# Subset Genotypes

This comes from the example notebook and the [tour of scikit-allel](http://alimanfoo.github.io/2016/06/10/scikit-allel-tour.html) blog post.

Here we're going ot subset genotypes. Most of the functionality comes from base numpy/pandas selection and filtering.

In [1]:
import numpy as np
import zarr
import pandas as pd
import dask.array as da
import allel
import scipy
from pprint import pprint
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('white')
sns.set_style('ticks')
sns.set_context('notebook')
%matplotlib inline

In [2]:
from dask_kubernetes import KubeCluster
cluster = KubeCluster(n_workers=30)
cluster

distributed.scheduler - INFO - Clear task state
/opt/conda/lib/python3.7/site-packages/distributed/dashboard/core.py:72: UserWarning: 
Port 8787 is already in use. 
Perhaps you already have a cluster running?
Hosting the diagnostics dashboard on a random port instead.
  warnings.warn("\n" + msg)
distributed.scheduler - INFO -   Scheduler at:   tcp://10.35.63.92:33501
distributed.scheduler - INFO -   dashboard at:                    :37053


## Import the Variant Data

In [3]:
import gcsfs

gcs_bucket_fs = gcsfs.GCSFileSystem(project='malariagen-jupyterhub', token='anon', access='read_only')

storage_path = 'ag1000g-release/phase2.AR1/variation/main/zarr/pass/ag1000g.phase2.ar1.pass'
store = gcsfs.mapping.GCSMap(storage_path, gcs=gcs_bucket_fs, check=False, create=False)
callset = zarr.Group(store)

In [4]:
chrom = '3L'

# All the variants
variants = allel.VariantChunkedTable(callset[chrom]['variants'], 
                                     names=['POS', 'REF', 'ALT', 'DP', 'MQ', 'QD'],
                                     index='POS')
variants

<VariantChunkedTable shape=(10640388,) dtype=[('POS', '<i4'), ('REF', 'S1'), ('ALT', 'S1', (3,)), ('DP', '<i4'), ('MQ', '<f4'), ('QD', '<f4')]
   nbytes=202.9M cbytes=-6 cratio=-35467960.0
   values=zarr.hierarchy.Group>

## Filter Variants

In [43]:
# Take a look at 003-Filter-Variants.ipyng for a more indepth look on filtering variants

def filter_variants(table, expression='(QD > 5) & (MQ > 40) & (DP > 15000) & (DP < 30000)'):
    """Filter a VariantChunkedTable based on a python expression"""
    selection = table.eval(expression)[:]
    return selection, table.compress(selection)

In [37]:
variant_selection, variants_pass = filter_variants(variants)
variants_pass

<VariantChunkedTable shape=(304050,) dtype=[('POS', '<i4'), ('REF', 'S1'), ('ALT', 'S1', (3,)), ('DP', '<i4'), ('MQ', '<f4'), ('QD', '<f4')]
   nbytes=5.8M cbytes=3.3M cratio=1.7
   values=allel.chunked.storage_zarr.ZarrTable>

In [38]:
variant_selection

array([False, False, False, ..., False, False, False])

## Subset genotypes
Now that we have some idea of variant quality, let’s look at our samples and at the genotype calls.

In [10]:
calldata = callset[chrom]['calldata']
calldata

<zarr.hierarchy.Group '/3L/calldata'>

In [11]:
list(calldata.keys())

['GT']

distributed.scheduler - INFO - Remove worker tcp://10.35.115.2:35233
distributed.core - INFO - Removing comms to tcp://10.35.115.2:35233


In [12]:
# In phase 1 data this is 
# genotypes = allel.GenotypeChunkedArray(calldata['genotype'])
genotypes = allel.GenotypeChunkedArray(calldata['GT'])
genotypes

<GenotypeChunkedArray shape=(10640388, 1142, 2) dtype=int8 chunks=(524288, 61, 2)
   nbytes=22.6G cbytes=-1 cratio=-24302646192.0
   compression=blosc compression_opts={'cname': 'zstd', 'clevel': 1, 'shuffle': -1, 'blocksize': 0}
   values=zarr.core.Array>

distributed.scheduler - INFO - Remove worker tcp://10.35.108.2:46359
distributed.core - INFO - Removing comms to tcp://10.35.108.2:46359
distributed.scheduler - INFO - Register tcp://10.35.96.2:39267
distributed.scheduler - INFO - Starting worker compute stream, tcp://10.35.96.2:39267
distributed.core - INFO - Starting established connection


N.B., at this point we have not loaded any data into memory, it is still in the Zarr file. 

(This comes from the blog post but references Phase1 data. We're using Phase 2)

From the representation above we have some diagnostic information about the genotypes, for example, we have calls for 16,437,135 variants in 765 samples with ploidy 2 (i.e., diploid). Uncompressed these data would be 23.4G but the data are compressed and so actually use 1.2G on disk.

We can also see genotype calls for the first 5 variants in the first and last 5 samples, which are all missing (“./.”).

Before we go any furter, let’s also pull in some data about the 765 samples we’ve genotyped.

In [13]:
!wget --no-clobber ftp://ngs.sanger.ac.uk/production/ag1000g/phase2/AR1/samples/samples.meta.txt

File ‘samples.meta.txt’ already there; not retrieving.


In [24]:
samples = pd.read_csv('samples.meta.txt', sep='\t')
samples.head()

array(['GHcol', 'GHgam', 'BFgam', 'BFcol', 'UGgam', 'GM', 'GW', 'KE',
       'CMgam', 'FRgam', 'GQgam', 'AOcol', 'GAgam', 'GNgam', 'GNcol',
       'CIcol'], dtype=object)

In [25]:
samples.population.unique()

array(['GHcol', 'GHgam', 'BFgam', 'BFcol', 'UGgam', 'GM', 'GW', 'KE',
       'CMgam', 'FRgam', 'GQgam', 'AOcol', 'GAgam', 'GNgam', 'GNcol',
       'CIcol'], dtype=object)

In [16]:
samples.population.value_counts()

CMgam    297
UGgam    112
BFgam     92
GW        91
AOcol     78
BFcol     75
CIcol     71
GAgam     69
GM        65
GHcol     55
KE        48
GNgam     40
FRgam     24
GHgam     12
GQgam      9
GNcol      4
Name: population, dtype: int64

Let’s work with two populations only for simplicity.

In [40]:
sample_selection = samples.population.isin({'GHcol', 'GHgam'}).values
sample_selection[:5]

array([ True,  True,  True,  True,  True])

Now restrict the samples table to only these two populations.

In [41]:
samples_subset = samples[sample_selection]
samples_subset.reset_index(drop=True, inplace=True)
samples_subset.head()

,ox_code,src_code,population,country,location,site,contributor,contact,year,m_s,sex,n_sequences,mean_coverage,ebi_sample_acc,latitude,longitude
0,AA0040-C,Twifo_Praso__E2,GHcol,Ghana,Twifo Praso,Twifo Praso,David Weetman,David Weetman,2012,M,F,95033368,30.99,ERS311878,5.60858,-1.54926
1,AA0041-C,Twifo_Praso__H3,GHcol,Ghana,Twifo Praso,Twifo Praso,David Weetman,David Weetman,2012,M,F,95843804,31.70,ERS311886,5.60858,-1.54926
2,AA0042-C,Takoradi_C7,GHcol,Ghana,Takoradi,Takoradi,David Weetman,David Weetman,2012,M,F,107420666,35.65,ERS311894,4.91217,-1.77397
3,AA0043-C,Takoradi_H8,GHcol,Ghana,Takoradi,Takoradi,David Weetman,David Weetman,2012,M,F,95993752,29.46,ERS311902,4.91217,-1.77397
4,AA0044-C,Takoradi_D10,GHcol,Ghana,Takoradi,Takoradi,David Weetman,David Weetman,2012,M,F,103044262,33.67,ERS311910,4.91217,-1.77397


Now let’s subset the genotype calls to keep only variants that pass our quality filters and only samples in our two populations of interest.

In [42]:
%%time
genotypes_subset = genotypes.subset(variant_selection, sample_selection)

CPU times: user 1min 37s, sys: 31 s, total: 2min 8s
Wall time: 1min 1s


In [44]:
genotypes_subset

<GenotypeChunkedArray shape=(304050, 67, 2) dtype=int8 chunks=(4751, 67, 2)
   nbytes=38.9M cbytes=2.7M cratio=14.4
   compression=blosc compression_opts={'cname': 'lz4', 'clevel': 5, 'shuffle': 1, 'blocksize': 0}
   values=zarr.core.Array>